In [1]:
import pandas as pd
import numpy as np

In [7]:
df=pd.read_csv("datasets/tcga/TCGA/mainTable.csv", index_col=0)

In [3]:
with open("corpus.txt",'w') as file:
    for sample in df.columns:
        for g in np.array(df[sample].sort_values(ascending=False).index[:1000],dtype=str):
            file.write(g)
            file.write(" ")
        file.write("\n")

In [4]:
del df

In [12]:
df_words = pd.read_csv("topicmapping/TCGA/word_wn_count.txt", sep=' ', header=None)
df_words.columns=['word', 'word-id', 'occurrence']
df_words.sort_values('word-id')

,word,word-id,occurrence
6374,ENSG00000202198,0,309
9814,ENSG00000245532,1,8547
8919,ENSG00000133392,2,2644
9986,ENSG00000124942,3,10424
10175,ENSG00000156508,4,11093
...,...,...,...
720,ENSG00000126870,10176,1
721,ENSG00000185418,10177,1
722,ENSG00000107371,10178,1
723,ENSG00000235169,10179,1


In [13]:
df_topic_distr = pd.read_csv("topicmapping/TCGA/lda_gammas_final.txt", sep=' ', header=None)
df_topic_distr.columns=['Topic %d'%(t+1) for t in df_topic_distr.columns]
df_topic_distr.index.name='i_doc'
df_topic_distr.insert(0,'doc',df.columns)
df_topic_distr=df_topic_distr.dropna(how='all',axis=1)
df_topic_distr

,doc,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5
i_doc,,,,,,
0,a62cfb55-6820-404c-959b-e33a2f732f25.htseq.cou...,162.099000,0.095279,148.970,0.043666,689.164000
1,d32c878b-e693-4eae-a615-7767371b3ae4.htseq.cou...,262.581000,737.656000,0.039,0.043666,0.051386
2,3bc29a02-86b5-400e-be79-e6c76552d52b.htseq.cou...,289.626000,634.389000,0.039,0.043666,76.273900
3,e3402ce9-c06b-48e2-b5f4-ba81d3e1dac7.htseq.cou...,542.649000,0.095279,405.994,0.043666,51.590100
4,2117d7f3-ed36-4083-aa33-6a231a5ff4cd.htseq.cou...,0.142196,559.286000,370.278,0.043666,70.621700
...,...,...,...,...,...,...
11088,875c2a27-9732-4b8a-affc-8ea591595a43.htseq.cou...,0.142196,1000.100000,0.039,0.043666,0.051386
11089,b667a6ae-353f-4045-a442-a4320231ed73.htseq.cou...,443.884000,0.095279,0.039,0.043666,556.310000
11090,41263b0e-e2db-4ecf-8152-136364e1ad66.htseq.cou...,423.347000,0.095279,0.039,0.043666,576.847000


In [14]:
clusters = df_topic_distr.drop('doc',1).values.argmax(1)
df_clusters=pd.DataFrame()
for cluster in range(np.max(clusters)+1):
    elems=df.columns[clusters==cluster].values
    df_clusters.insert(0,'Cluster %d'%(cluster+1),np.concatenate([elems, ['' for _ in range(len(df.columns)- len(elems))]]))

In [15]:
df_clusters.sort_index(axis=1).to_csv("tm_level_0_clusters.csv", index=False, header=True)

In [16]:
df_topic_distr.to_csv("tm_level_0_topic-dist.csv", index=True, header=True)